# <center> Innoscape - Telecom Paristech - Projet fil rouge </center>
# <center> Mapping Groupes de produits </center>

In [2]:
# Librairies 
import pandas as pd
import numpy as np
from scipy import stats as scs
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import re
pd.options.mode.chained_assignment = None

chemin = "C:/Users/JN/Desktop/Telecom PT/Projet fil rouge/github/INNOSCAPE/Data/"
#chemin = "C:/Users/Pascal/Desktop/Telecom ParisTech/Cours/INFMDI780 - Projet BigData/Data/"

## Importation des données et reformatage

In [3]:
# Import data source
pd.set_option('display.max_columns', 60)
df = pd.read_csv(chemin + "t3_all_fr_prices_posweekly_prod_without_price.csv", sep = ",")

#Création d'un df allégé pour le travail - on ne retient que les colonnes identifiées comme utiles pour l'analyse à ce stade
columns_to_keep = ['t3_all_fr_prices_posweekly_prod.productfamily_seller_standard',
                   't3_all_fr_prices_posweekly_prod.productseller_name_standard',
                   't3_all_fr_prices_posweekly_prod.id_source',
                   't3_all_fr_prices_posweekly_prod.prodcategory4_seller_standard']
df2 = df[columns_to_keep]

#on raccourcit un peu les noms des colonnes en ôtant les préfixes redondants
shortened_names = map(lambda string: string.replace('t3_all_fr_prices_posweekly_prod.',''), columns_to_keep)
col_name_dict = dict(zip(columns_to_keep, shortened_names))
df2.rename(columns=col_name_dict, inplace=True)

# Import data typo INNOSCAPE
pd.set_option('display.max_columns', 60)
typo = pd.read_csv(chemin + "2018.11.21_innoscape_internal_architecture.csv")
typo.drop("Date", axis = 1, inplace = True)


In [4]:
# Retrait des caractères spéciaux et de la ponctuation
dictionary = {'à':'a','â':'a','ä':'a',
              'é':'e','è':'e','ê':'e','ë':'e',
              'ï':'i','î':'i',
              'ô':'o','ö':'o',
              'ù':'u','û':'u',
              'ç':'c',
              ',':' ',"'":" ",";":" ",":":" ","\?":" ","!":" ","\.":'',
              "\(":"","\)":"","\[":"","\]":"","-":" ","\+":"","/":"","\*":"","–":" ",
              "&":"et"
              }

typo['innoscape_ref_productfamily_seller'].replace(dictionary, regex=True, inplace=True)
df2.replace(dictionary, regex=True, inplace=True)
#Attention! Ne pas chercher à faire str.replace(".","") ==> efface tout!

#passage en minuscules
typo['innoscape_ref_productfamily_seller'] = typo['innoscape_ref_productfamily_seller'].str.lower()
df2['productfamily_seller_standard'] = df2['productfamily_seller_standard'].str.lower()
df2['productseller_name_standard'] = df2['productseller_name_standard'].str.lower()

df2.head()

productfamily_seller_standard  \
0            colles   carrelage   
1                           NaN   
2                           NaN   
3                        mastic   
4                         cires   

                         productseller_name_standard id_source  \
0  sikaceram xtra   colle a carrelage interieur e...    AMZ_FR   
1              ariston  scaldabagno elettrico pro  h    AMZ_FR   
2  aeg panier à couverts lave vaisselle cage et h...    AMZ_FR   
3              colle mastic pour joints d etancheite    AMZ_FR   
4  sikagard protection sol mat   impermeabilisant...    AMZ_FR   

  prodcategory4_seller_standard  
0            colles   carrelage  
1                           NaN  
2                           NaN  
3                        mastic  
4                         cires

In [5]:
#même démarche avec les stop words
#pour éviter des suppressions aberrantes (par ex: supprimer tous les 'a'), on va d'abord scinder les champs texte en 
#listes de mots

df2['prod_fam'] = df2.productfamily_seller_standard.str.split()
df2['description'] = df2.productseller_name_standard.str.split()
df2['cat4'] = df2.prodcategory4_seller_standard.str.split()
typo['prod_fam'] = typo['innoscape_ref_productfamily_seller'].str.split()

#on supprime également les entrées vides dans df2:
df2 = df2.dropna()

#on va ensuite pouvoir retirer les stop words:
stop_words = {'a',
             'ai',
             'aie',
             'aient',
             'aies',
             'ait',
             'as',
             'au',
             'aura',
             'aurai',
             'auraient',
             'aurais',
             'aurait',
             'auras',
             'aurez',
             'auriez',
             'aurions',
             'aurons',
             'auront',
             'aux',
             'avaient',
             'avais',
             'avait',
             'avec',
             'avez',
             'aviez',
             'avions',
             'avons',
             'ayant',
             'ayante',
             'ayantes',
             'ayants',
             'ayez',
             'ayons',
             'c',
             'ce',
             'ces',
             'd',
             'dans',
             'de',
             'des',
             'du',
             'elle',
             'en',
             'es',
             'est',
             'et',
             'eu',
             'eue',
             'eues',
             'eurent',
             'eus',
             'eusse',
             'eussent',
             'eusses',
             'eussiez',
             'eussions',
             'eut',
             'eux',
             'eûmes',
             'eût',
             'eûtes',
             'furent',
             'fus',
             'fusse',
             'fussent',
             'fusses',
             'fussiez',
             'fussions',
             'fut',
             'fûmes',
             'fût',
             'fûtes',
             'il',
             'j',
             'je',
             'l',
             'la',
             'le',
             'leur',
             'lui',
             'm',
             'ma',
             'mais',
             'me',
             'mes',
             'moi',
             'mon',
             'même',
             'n',
             'ne',
             'nos',
             'notre',
             'nous',
             'on',
             'ont',
             'ou',
             'par',
             'pas',
             'pour',
             'qu',
             'que',
             'qui',
             's',
             'sa',
             'se',
             'sera',
             'serai',
             'seraient',
             'serais',
             'serait',
             'seras',
             'serez',
             'seriez',
             'serions',
             'serons',
             'seront',
             'ses',
             'soient',
             'sois',
             'soit',
             'sommes',
             'son',
             'sont',
             'soyez',
             'soyons',
             'suis',
             'sur',
             't',
             'ta',
             'te',
             'tes',
             'toi',
             'ton',
             'tu',
             'un',
             'une',
             'vos',
             'votre',
             'vous',
             'y',
             'à',
             'étaient',
             'étais',
             'était',
             'étant',
             'étante',
             'étantes',
             'étants',
             'étiez',
             'étions',
             'été',
             'étée',
             'étées',
             'étés',
             'êtes',
             ',',
             '(',
             ')',
             '[',
             ']',
             '-',
             '/',
             '+',
             '*',
             '{',
             '}',
             '!',
             ';',
             ':',
             '?',
             '.',
             '&'
             }

def clean_stop_words(row, col_title): 
    return set(row[col_title]) - stop_words

typo['prod_fam'] = typo.apply(clean_stop_words, args=(['prod_fam']), axis=1)
df2['prod_fam'] = df2.apply(clean_stop_words, args=(['prod_fam']), axis=1)
df2['description'] = df2.apply(clean_stop_words, args=(['description']), axis=1)
df2['cat4'] = df2.apply(clean_stop_words, args=(['cat4']), axis=1)

In [6]:
typo[['innoscape_ref_productfamily_seller','prod_fam']].head(40)

innoscape_ref_productfamily_seller  \
0                             accessoires gros oeuvre   
1                              accessoires couverture   
2                              accessoires couverture   
3                                  divers gros oeuvre   
4                gouttiere  descente et accessoires     
5   poudre beton  chaux  ciment  enduit  mortier  ...   
6                gouttiere  descente et accessoires     
7                            accessoires chauffe eau    
8                                         adoucisseur   
9                                     aerateur et vmc   
10                                           arrosage   
11                                        anti mousse   
12                                   mousse expansive   
13                 accessoires outillage stationnaire   
14                             outillage stationnaire   
15           brise vue  canisse  brande et brise vent   
16          aspirateurs  nettoyeurs hp et accessoires   
17                        compresseurs et accessoires   
18               gouttiere  descente et accessoires     
19                                           arrosage   
20                                        adoucisseur   
21                             accessoires couverture   
22                            chauffe eau electrique    
23                            chauffe eau electrique    
24                                    chauffe eau gaz   
25                        chauffe eau thermodynamique   
26                           mastic colle etancheite    
27                                             divers   
28                  conduit et accessoires d aeration   
29                      pistolet a colle et agrafeuse   
30  accessoires et consommables outillage electrop...   
31                               batterie et chargeur   
32                                         divers oep   
33                  defonceuse et lamelleuse et rabot   
34                           mastic colle etancheite    
35                           mastic colle etancheite    
36                           mastic colle etancheite    
37                           mastic colle etancheite    
38                      fixation  pipe et joint de wc   
39               gouttiere  descente et accessoires     

                                             prod_fam  
0                         {accessoires, gros, oeuvre}  
1                           {couverture, accessoires}  
2                           {couverture, accessoires}  
3                              {divers, gros, oeuvre}  
4                  {gouttiere, descente, accessoires}  
5   {ciment, beton, ragreage, enduit, mortier, pou...  
6                  {gouttiere, descente, accessoires}  
7                         {eau, chauffe, accessoires}  
8                                       {adoucisseur}  
9                                     {vmc, aerateur}  
10                                         {arrosage}  
11                                     {mousse, anti}  
12                                {mousse, expansive}  
13             {outillage, accessoires, stationnaire}  
14                          {outillage, stationnaire}  
15                {canisse, brande, brise, vent, vue}  
16         {hp, aspirateurs, nettoyeurs, accessoires}  
17                        {compresseurs, accessoires}  
18                 {gouttiere, descente, accessoires}  
19                                         {arrosage}  
20                                      {adoucisseur}  
21                          {couverture, accessoires}  
22                         {eau, chauffe, electrique}  
23                         {eau, chauffe, electrique}  
24                                {eau, chauffe, gaz}  
25                    {eau, thermodynamique, chauffe}  
26                        {colle, etancheite, mastic}  
27                                           {divers}  
28                   {conduit, aeration, accessoires}  
2

## Deuxième tentative de mapping des familles de produits

In [7]:
#Dans df2, on concatène les mots de la famille et de la description produit
# def concat(row):
#     return row['prod_fam'] | row['description']

def concat(row, col1, col2):
    return row[col1] | row[col2] 

df2['fam+descr'] = df2.apply(concat, args=('prod_fam', 'description'), axis = 1)
df2.head(20)

productfamily_seller_standard  \
0                      colles   carrelage   
3                                  mastic   
4                                   cires   
6                       perceuse visseuse   
7                   produits d etancheite   
8                               arroseurs   
11                  pompes pour fontaines   
12                   secateurs et ciseaux   
13    accessoires pour outillage sans fil   
14  pieces et accessoires pour chaudieres   
16     filtres pour aspirateurs portatifs   
17                      meuleuses d angle   
19                               conduits   
20                               conduits   
21                        mastic au latex   
22                     pieces et raccords   
23                   casques et ecouteurs   
24                   bordures pour jardin   
25                    marteau perforateur   
26                          tron onneuses   

                          productseller_name_standard id_source  \
0   sikaceram xtra   colle a carrelage interieur e...    AMZ_FR   
3               colle mastic pour joints d etancheite    AMZ_FR   
4   sikagard protection sol mat   impermeabilisant...    AMZ_FR   
6   perceuse visseuse ryobi  oneplus    batterie l...    AMZ_FR   
7   sika multiseal   bande d etancheite autocollan...    AMZ_FR   
8                  black and decker  arroseur impulse    AMZ_FR   
11          nortene  pompe fontaine  h  cable  m noir    AMZ_FR   
12  black and decker    secateur longueur    cmdia...    AMZ_FR   
13                       ryobi   massicot a carrelage    AMZ_FR   
14                         securite gaz pochette ref     AMZ_FR   
16  aeg  allergy plus filtre lavable pour aeg et p...    AMZ_FR   
17              aeg   meuleuse électrique sans fil  v    AMZ_FR   
19  ariston thermo  nuos kit manchon diametre   blanc    AMZ_FR   
20  ariston thermo  nuos grille ressorts trou diam...    AMZ_FR   
21  sikaflex  fc   mastic colle multi usages et mu...    AMZ_FR   
22        black and decker  raccord de serrage triple    AMZ_FR   
23  imtradex bundle headset incl noisehelper  basi...    AMZ_FR   
24  windhager  brise vue en brande de bruyere marr...    AMZ_FR   
25  aeg powertools marteau perforateur bbh  pro li...    AMZ_FR   
26  ryobi  podadora chaine  mm larg  cm diametre c...    AMZ_FR   

            prodcategory4_seller_standard  \
0                      colles   carrelage   
3                                  mastic   
4                                   cires   
6                               perceuses   
7                   produits d etancheite   
8                               arroseurs   
11                                 pompes   
12                   secateurs et ciseaux   
13    accessoires pour outillage sans fil   
14  pieces et accessoires pour chaudieres   
16                                filtres   
17                              meuleuses   
19                               conduits   
20                               conduits   
21                                 mastic   
22                  tuyaux et accessoires   
23                   casques et ecouteurs   
24                   bordures pour jardin   
25                              perceuses   
26                          tron onneuses   

                               prod_fam  \
0                   {carrelage, colles}   
3                              {mastic}   
4                               {cires}   
6                  {visseuse, perceuse}   
7                {etancheite, produits}   
8                           {arroseurs}   
11                  {fontaines, pompes}   
12                 {ciseaux, secateurs}   
13  {sans, outillage, accessoires, fil}   
14    {chaudieres, pieces, accessoires}   
16    {aspirateurs, portatifs, filtres}   
17                   {meuleuses, angle}   
19                           {conduits}   
20                           {conduits}   
21                      {mastic, latex}   
22                   {pi

In [8]:
# On se place sur un échantillon test de 1000 lignes de la base df2
import random

test_indices = np.array(random.sample(list(df2.index.values),1000)).astype(int)
df_test = df2.loc[test_indices]

#on crée un dictionnaire {familles_Innoscape, index_des_familles_LM}, afin de pouvoir tenir compte du fait
#que plusieurs familles LM ont été regroupées dans certaines familles Innoscape
fam_dict = dict(zip(typo.innoscape_ref_productfamily_seller, typo.index.values))

#on calcule ensuite, pour chaque famille de la typologie Innoscape, et pour chaque produit de l'échantillon test, *
#la similitude de Jaccard:

for element in typo.innoscape_ref_productfamily_seller.unique():
    col_title = "similarity "+str(element)
    
    def jaccard(row):
        return len(row['fam+descr']&typo.loc[fam_dict[element],'prod_fam']) / len(row['fam+descr']|typo.loc[fam_dict[element],'prod_fam'])
    df_test[col_title] = df_test.apply(jaccard, axis=1)

def get_max_jaccard(row):
    return np.max([row["similarity "+str(element)] for element in typo.innoscape_ref_productfamily_seller.unique()])

def get_argmax_jaccard(row):
    return np.argmax([row["similarity "+str(element)] for element in typo.innoscape_ref_productfamily_seller.unique()])

df_test['max_jaccard'] = df_test.apply(get_max_jaccard, axis=1)
df_test['argmax_jaccard'] = df_test.apply(get_argmax_jaccard, axis=1)


In [9]:
df2.loc[test_indices]

productfamily_seller_standard  \
284                      meuleuses droites   
1972                chauffe eau electrique   
2268               accessoires chauffe eau   
1505          evacuation des eaux de pluie   
1768                            couverture   
348                    tondeuses rotatives   
4173                         gouttiere pvc   
173                     climatiseurs split   
1529                            couverture   
2287          evacuation des eaux de pluie   
1968                            couverture   
1450                            couverture   
1808                            couverture   
2251                            chaudieres   
1466          evacuation des eaux de pluie   
1580          evacuation des eaux de pluie   
1844                chauffe eau electrique   
2290          evacuation des eaux de pluie   
555                        scies sauteuses   
3704              etancheite de la toiture   
3856              etancheite de la toiture   
2414          evacuation des eaux de pluie   
2370          evacuation des eaux de pluie   
236                      perceuse visseuse   
128                             agrafeuses   
1983  etancheite des toitures et terrasses   
2214                            couverture   
2141                            couverture   
49                      colles   carrelage   
585                     contacteurs   cles   
...                                    ...   
1745                            couverture   
767               lames de scie circulaire   
31                         mastic au latex   
1476                            couverture   
2369                            couverture   
3624                         gouttiere pvc   
340                            humidimetre   
562                                 mastic   
4029              etancheite de la toiture   
1777                            couverture   
2050                            couverture   
1703          evacuation des eaux de pluie   
628                               materiel   
3952              etancheite de la toiture   
2408                            couverture   
3666              etancheite de la toiture   
2324                            couverture   
1896                            couverture   
2450                              ragreage   
1563                            couverture   
1987                chauffe eau electrique   
2106                            couverture   
1644                            couverture   
100                       visseuses   choc   
520                                  gazon   
1530           chauffe eau thermodynamique   
4037                         gouttiere pvc   
1814                            couverture   
1386  etancheite des toitures et terrasses   
741                     filets pour jardin   

                            productseller_name_standard  id_source  \
284            ryobi rag   meuleuse électrique a fil  w     AMZ_FR   
1972  chauffe eau a resistance mono steatite "olympic"   BRCMAN_FR   
2268             reducteur de pression pour chauffe eau  BRCMAN_FR   
1505                      coude pvc ardoise   diam    °  BRCMAN_FR   
1768            durovent tuile a douille dbl romane rge  BRCMAN_FR   
348   ryobi tondeuse electrique  w   cm largeur de c...     AMZ_FR   
4173                      descente de gouttiere ø gris    CASTO_FR   
173   ariston thermo  unite interieure modele alys e...     AMZ_FR   
1529                         crochet pr tuiledble tuile  BRCMAN_FR   
2287        angle mixte pvc blanc  a joint   developpe   BRCMAN_FR   
1968                       rive a rabat gauche  ardoise  BRCMAN_FR   
1450             tuile chatiere galleane  rouge vieilli  BRCMAN_FR   
1808                faitiere demi ronde de  rge vieilli  BRCMAN_FR   
2251               chaudiere a bois et charbon en fonte  BRCMAN_FR   
1466  gouttiere demi ronde pvc girpi developpe  sabl...  BRCMAN_FR   
1580             recuperateur d eau de p

In [10]:
#recensement des lignes ayant effectivement pu être rapprochées de catégories Innoscape
matches = df_test[df_test['max_jaccard']>0]

print("On constate que %s lignes sur 1000 partagent au moins un mot en commun avec une des familles produits Innoscape." %str(len(matches.index.values)))

On constate que 887 lignes sur 1000 partagent au moins un mot en commun avec une des familles produits Innoscape.


In [11]:
matches['max_jaccard'].describe()

count    887.000000
mean       0.180276
std        0.099960
min        0.041667
25%        0.125000
50%        0.166667
75%        0.200000
max        0.750000
Name: max_jaccard, dtype: float64

In [12]:
#affichage de recoupements effectués:
matches[['productfamily_seller_standard','productseller_name_standard','prod_family_final','max_jaccard']]

KeyError: "['prod_family_final'] not in index"

En regardant plus en détail, on constate cependant que certaines correspondances laissent à désirer:

In [13]:
dict(zip(matches.productfamily_seller_standard, matches.prod_family_final))

AttributeError: 'DataFrame' object has no attribute 'prod_family_final'

## On applique cette démarche aux lignes Leroy-Merlin Exclusivement

In [14]:
df_LM = df2[df2['id_source']=='LM_FR']
nLM = len(df_LM.index.values)

print("Nb de lignes Leroy-Merlin : %s" %str(nLM))
print()

#on calcule ensuite, pour chaque famille de la typologie Innoscape, et pour chaque produit de l'échantillon test, *
#la similitude de Jaccard:

for element in typo.innoscape_ref_productfamily_seller.unique():
    col_title = "similarity "+str(element)
    
    def jaccard(row):
        return len(row['fam+descr']&typo.loc[fam_dict[element],'prod_fam']) / len(row['fam+descr']|typo.loc[fam_dict[element],'prod_fam'])
    df_LM[col_title] = df_LM.apply(jaccard, axis=1)

def get_max_jaccard(row):
    return np.max([row["similarity "+str(element)] for element in typo.innoscape_ref_productfamily_seller.unique()])

def get_argmax_jaccard(row):
    return np.argmax([row["similarity "+str(element)] for element in typo.innoscape_ref_productfamily_seller.unique()])

df_LM['max_jaccard'] = df_LM.apply(get_max_jaccard, axis=1)
df_LM['argmax_jaccard'] = df_LM.apply(get_argmax_jaccard, axis=1)

df_LM['prod_family_final'] = df_LM.apply(get_category, axis = 1)

#recencement des lignes ayant effectivement pu être rapprochées de catégories Innoscape
matches_LM = df_LM[df_LM['max_jaccard']>0]
n_matches  = len(matches_LM.index.values)

print("On constate que %f pour cent des lignes Leroy partagent au moins un mot en commun avec une des familles produits Innoscape." %(n_matches/nLM*100))

Nb de lignes Leroy-Merlin : 9



NameError: name 'get_category' is not defined

Examinons les résultats plus en détail:

In [15]:
matches_LM['max_jaccard'].describe()

NameError: name 'matches_LM' is not defined

In [16]:
#affichage de recoupements effectués:
matches_LM[['productfamily_seller_standard','productseller_name_standard','prod_family_final','max_jaccard']]

NameError: name 'matches_LM' is not defined

# Variante : Mapping sur la famille puis sur la description

Voyons s'il est possible d'améliorer les résultats de la démarche ci-dessus en effectuant les recoupements en deux temps:
    - d'abord sur la base des noms de familles produits des vendeurs,
    - puis, si aucun regroupement n'est trouvé, à l'aide des descriptions de produits

In [17]:
#on reprend le même échantillon test de 1000 lignes prises au hasard que dans la partie précédente:
df_test = df2.loc[test_indices]

In [18]:
df_test.head()

productfamily_seller_standard  \
284              meuleuses droites   
1972        chauffe eau electrique   
2268       accessoires chauffe eau   
1505  evacuation des eaux de pluie   
1768                    couverture   

                            productseller_name_standard  id_source  \
284            ryobi rag   meuleuse électrique a fil  w     AMZ_FR   
1972  chauffe eau a resistance mono steatite "olympic"   BRCMAN_FR   
2268             reducteur de pression pour chauffe eau  BRCMAN_FR   
1505                      coude pvc ardoise   diam    °  BRCMAN_FR   
1768            durovent tuile a douille dbl romane rge  BRCMAN_FR   

     prodcategory4_seller_standard                     prod_fam  \
284                      meuleuses         {meuleuses, droites}   
1972        chauffe eau electrique   {eau, chauffe, electrique}   
2268       accessoires chauffe eau  {eau, chauffe, accessoires}   
1505  evacuation des eaux de pluie    {evacuation, pluie, eaux}   
1768                    couverture                 {couverture}   

                                            description  \
284          {rag, meuleuse, électrique, w, fil, ryobi}   
1972  {"olympic", chauffe, steatite, eau, mono, resi...   
2268                {reducteur, pression, eau, chauffe}   
1505                     {ardoise, diam, °, coude, pvc}   
1768       {tuile, durovent, dbl, douille, rge, romane}   

                             cat4  \
284                   {meuleuses}   
1972   {eau, chauffe, electrique}   
2268  {eau, chauffe, accessoires}   
1505    {evacuation, pluie, eaux}   
1768                 {couverture}   

                                              fam+descr  
284   {rag, meuleuse, électrique, w, droites, fil, m...  
1972  {chauffe, "olympic", electrique, steatite, eau...  
2268   {reducteur, pression, accessoires, eau, chauffe}  
1505  {ardoise, diam, eaux, pluie, °, coude, pvc, ev...  
1768  {tuile, couverture, durovent, dbl, rge, douill...

On reprend le travail précédent, en se concentrant seulement sur les noms de familles dans un premier temps. On redéfinit pour cela les fonctions utilisées précédemment, pour factoriser le code:

In [19]:
#on calcule ensuite, pour chaque famille de la typologie Innoscape, et pour chaque produit de l'échantillon test, *
#la similitude de Jaccard:

def jaccard(row_df1, col_df1, df2, idx_df2, col_df2):
    """ fonction à appliquer sur un dataframe, noté ici df1
    calcule la distance de Jaccard entre la colonne col_df1 du dataframe df1,
    et une certaine cellule située à l'index idx_df2, dans la colonne col_df2 d'un autre dataframe noté df2
    """
    return len(row_df1[col_df1]&df2.loc[idx_df2,col_df2]) / len(row_df1[col_df1]|df2.loc[idx_df2,col_df2])

def jaccard_matrix(df, col_df_tokenized, typo, col_typo, col_typo_tokenized, string_header):
    """ 
    prend un DataFrame (df) en input, et donne en output une copie de df avec une série de colonnes
    contenant les distances de jaccard avec chaque famille d'une typologie produits (typo).
    
    df: DataFrame: base produits obtenue par crawling et préprocessée
    col_df_tokenized: string: nom de la colonne de df où se trouvent les données à comparer (tokenisées)
    typo: DataFrame: contient typologie produits Innoscape
    col_typo: str: colonne de typo où se trouvent les données à comparer (texte brut)
    col_typo_tokenized: colonne de typo où se trouvent les données à comparer (tokénisées)
    string_header: préfixe à mettre dans les noms des colonnes ajoutées
    """
    
    df_copy = df.copy()
    for element in typo[col_typo].unique():
        col_title = str(string_header)+'_'+str(element)
        fam_dict = dict(zip(typo[col_typo], typo.index.values))
        df_copy[col_title] = df_copy.apply(jaccard,
                                           args=(col_df_tokenized, typo, fam_dict[element], col_typo_tokenized),
                                           axis=1)
    return df_copy

def get_max_similarity(row, string_header):
    """trouve, pour sur ligne donnée, le maximum des valeurs prises sur un ensemble de cellules.
    Les titres des colonnes doivent être de la forme "string_header_element", où string_header est
    un nom générique passé en argument, et "element" une famille de produits Innoscape
    """
    return np.max([row[string_header+'_'+str(element)] for element in typo.innoscape_ref_productfamily_seller.unique()])

def get_argmax_similarity(row, string_header):
    """Renvoie l'indice de la colonne contenant la plus grande distance de Jaccard, pour une ligne donnée.
    Les titres des colonnes doivent être de la forme "string_header_element", où string_header est
    un nom générique passé en argument, et "element" une famille de produits Innoscape

    """
    return np.argmax([row[string_header+'_'+str(element)] for element in typo.innoscape_ref_productfamily_seller.unique()])

# on exprime enfin explicitement les correspondances trouvées:
cat_dict = dict(zip(range(len(typo.innoscape_ref_productfamily_seller.unique())),typo.innoscape_ref_productfamily_seller.unique()))

def get_category(row, col_title, max_col_name):
    """ renvoie la famille Innoscape correspondant à l'indice stocké dans la colonne col_title
    col_title: string
    max_col_name: str: nom de la colonne où est stockée la similarité maximale donnée par gat_max_similarity
    """
    if row[max_col_name] == 0.0:
        return "no match"
    else:
        return cat_dict[row[col_title]]

On reprend ensuite la démarche réalisée précédemment sur les noms de familles produits:

In [20]:
df_test = df2.loc[test_indices]
df_test2 = jaccard_matrix(df_test, "prod_fam", typo, "innoscape_ref_productfamily_seller" , "prod_fam", "similarity_prod_fam")
df_test2['max_sim_prod_fam'] = df_test2.apply(get_max_similarity, args=(["similarity_prod_fam"]), axis=1)
df_test2['argmax_sim_prod_fam'] = df_test2.apply(get_argmax_similarity, args=(["similarity_prod_fam"]), axis=1)
df_test2['best_match_prod_fam'] = df_test2.apply(get_category, args=('argmax_sim_prod_fam','max_sim_prod_fam'), axis=1)
matching_prod_fam = df_test2[['productfamily_seller_standard','best_match_prod_fam', 'max_sim_prod_fam']]

Essayons de segmenter cet échantillon en fonction de la valeur de max_jaccard:

In [21]:
bucket0 = matching_prod_fam[matching_prod_fam['max_sim_prod_fam']==0].dropna()
bucket1 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0) & (matching_prod_fam['max_sim_prod_fam']<0.2)).dropna()
bucket2 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0.2) & (matching_prod_fam['max_sim_prod_fam']<0.4)).dropna()
bucket3 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0.4) & (matching_prod_fam['max_sim_prod_fam']<0.6)).dropna()
bucket4 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0.6) & (matching_prod_fam['max_sim_prod_fam']<0.8)).dropna()
bucket5 = matching_prod_fam.where((matching_prod_fam['max_sim_prod_fam']>0.8)).dropna()

buckets = [bucket0, bucket1, bucket2, bucket3, bucket4, bucket5]

Voyons, catégorie par catégorie, dans quelle mesure les regroupements sont pertinents:

In [22]:
bucket5.head(40)

productfamily_seller_standard       best_match_prod_fam  max_sim_prod_fam
1972        chauffe eau electrique   chauffe eau electrique                1.0
2268       accessoires chauffe eau  accessoires chauffe eau                1.0
1768                    couverture                couverture               1.0
1529                    couverture                couverture               1.0
1968                    couverture                couverture               1.0
1450                    couverture                couverture               1.0
1808                    couverture                couverture               1.0
1844        chauffe eau electrique   chauffe eau electrique                1.0
2214                    couverture                couverture               1.0
2141                    couverture                couverture               1.0
1749       accessoires chauffe eau  accessoires chauffe eau                1.0
1531                    couverture                couverture               1.0
2081       accessoires chauffe eau  accessoires chauffe eau                1.0
2023                    couverture                couverture               1.0
1874                    couverture                couverture               1.0
2198        chauffe eau electrique   chauffe eau electrique                1.0
1770                    couverture                couverture               1.0
1630                    couverture                couverture               1.0
1907                    couverture                couverture               1.0
2212                    couverture                couverture               1.0
2356                    couverture                couverture               1.0
1640                    couverture                couverture               1.0
1835                    couverture                couverture               1.0
1707                    couverture                couverture               1.0
1588                    couverture                couverture               1.0
1582                    couverture                couverture               1.0
2385                    couverture                couverture               1.0
1868                    couverture                couverture               1.0
1829                    couverture                couverture               1.0
1467                    couverture                couverture               1.0
1954                    couverture                couverture               1.0
1494                    couverture                couverture               1.0
1469                    couverture                couverture               1.0
1742                    couverture                couverture               1.0
124              chauffe eau   gaz           chauffe eau gaz               1.0
2217                    couverture                couverture               1.0
2353                    couverture                couverture               1.0
2283                    couverture                couverture               1.0
2209                    couverture                couverture               1.0
2262                    couverture                couverture               1.0

In [ ]:
bucket4.head(40)

In [ ]:
bucket3.head(40)

In [23]:
bucket2.head(40)

productfamily_seller_standard  \
4173                             gouttiere pvc   
555                            scies sauteuses   
3704                  etancheite de la toiture   
3856                  etancheite de la toiture   
598       brise vue et pare soleil pour jardin   
120   aspirateurs balais et balais electriques   
159                                     mastic   
3861                             gouttiere pvc   
667                          packs de batterie   
569       brise vue et pare soleil pour jardin   
3850                             gouttiere pvc   
811                              kits d outils   
2434                             scie sauteuse   
3761                             gouttiere pvc   
194                          packs de batterie   
449                              kits d outils   
649                          packs de batterie   
844                                     mastic   
208                     accessoires pour scies   
3774                             gouttiere pvc   
87                           packs de batterie   
665                       secateurs et ciseaux   
4170                             gouttiere pvc   
47    aspirateurs balais et balais electriques   
591             kit d arrosage goutte   goutte   
3857                  etancheite de la toiture   
577                  scies circulaires   table   
451                         compresseurs d air   
4155                             gouttiere pvc   
4057                  etancheite de la toiture   
4056                  etancheite de la toiture   
254                                     mastic   
3954                             gouttiere pvc   
4015                  etancheite de la toiture   
290                                   broyeurs   
730                                     mastic   
21                             mastic au latex   
490                                 souffleurs   
4086                  etancheite de la toiture   
3822                             gouttiere pvc   

                           best_match_prod_fam  max_sim_prod_fam  
4173      gouttiere  descente et accessoires            0.250000  
555                          scies electriques          0.333333  
3704                  mastic colle etancheite           0.250000  
3856                  mastic colle etancheite           0.250000  
598   brise vue  canisse  brande et brise vent          0.250000  
120                          scies electriques          0.250000  
159                   mastic colle etancheite           0.333333  
3861      gouttiere  descente et accessoires            0.250000  
667                       batterie et chargeur          0.333333  
569   brise vue  canisse  brande et brise vent          0.250000  
3850      gouttiere  descente et accessoires            0.250000  
811                      outils multifonctions          0.333333  
2434                               scie a main          0.333333  
3761      gouttiere  descente et accessoires            0.250000  
194                       batterie et chargeur          0.333333  
449                      outils multifonctions          0.333333  
649                       batterie et chargeur          0.333333  
844                   mastic colle etancheite           0.333333  
208                     accessoires couverture          0.333333  
3774      gouttiere  descente et accessoires            0.250000  
87                        batterie et chargeur          0.333333  
665                 cutter  scalpel et ciseaux          0.250000  
4170      gouttiere  descente et accessoires            0.250000  
47                           scies electriques          0.250000  
591                                   arrosage          0.333333  
3857                  mastic colle etancheite           0.250000  
577                          scies electriques          0.250000  
451                compresseurs et accessoires          0.333333  
4155      gouttiere  de

On commence à observer quelques rapprochements erronés, par exemple "produits de traitement et accessoires piscine" avec "produits et traitement bois interieur" (distance de Jaccard de 0.33). On peut cependant envisager de faire baisser le nombre d'erreur en supprimant, au cours du preprocessing, des termes peu spécifiques (par exemple "produit" ou "électrique").

In [24]:
bucket1.head(40)

productfamily_seller_standard  \
678   pieces de remplacement pour tondeuses   
727          tondeuses poussees et tractees   
485                       lames de massicot   
753           sets d embouts pour tournevis   
2437                               ragreage   
2450                               ragreage   

                                    best_match_prod_fam  max_sim_prod_fam  
678     tondeuses electriques  thermiques et autoportee          0.166667  
727     tondeuses electriques  thermiques et autoportee          0.166667  
485        dalles et lames bois pour terrasse et jardin          0.166667  
753        tournevis cruciforme  aimante  de precision…          0.166667  
2437  poudre beton  chaux  ciment  enduit  mortier  ...          0.142857  
2450  poudre beton  chaux  ciment  enduit  mortier  ...          0.142857

On constate un certain nombre d'erreurs d'attribution, pour des similitudes de Jaccard inférieures à 0.15 environ. Dans la suite, on retiendra donc provisoirement ce seuil pour déclencher une deuxième vague de regroupements sur la base des descriptions produits. Voyons maintenant ce qui se passe quand on complète le rapprochement par noms de familles par le rapprochement à partir des descriptions de produits:

In [25]:
df_test2 = jaccard_matrix(df_test, "description", typo, "innoscape_ref_productfamily_seller" , "prod_fam", "similarity_descr")
df_test2['max_sim_descr'] = df_test2.apply(get_max_similarity, args=(["similarity_descr"]), axis=1)
df_test2['argmax_sim_descr'] = df_test2.apply(get_argmax_similarity, args=(["similarity_descr"]), axis=1)
df_test2['best_match_descr'] = df_test2.apply(get_category, args=('argmax_sim_descr','max_sim_descr'), axis=1)
matching_descr = df_test2[['productfamily_seller_standard','productseller_name_standard','best_match_descr', 'max_sim_descr']]

In [26]:
bucket0 = matching_descr[matching_descr['max_sim_descr']==0].dropna()
bucket1 = matching_descr.where((matching_descr['max_sim_descr']>0) & (matching_descr['max_sim_descr']<0.05)).dropna()
bucket2 = matching_descr.where((matching_descr['max_sim_descr']>0.05) & (matching_descr['max_sim_descr']<0.1)).dropna()
bucket3 = matching_descr.where((matching_descr['max_sim_descr']>0.1) & (matching_descr['max_sim_descr']<0.15)).dropna()
bucket4 = matching_descr.where((matching_descr['max_sim_descr']>0.15) & (matching_descr['max_sim_descr']<0.2)).dropna()
bucket5 = matching_descr.where((matching_descr['max_sim_descr']>0.2)).dropna()

buckets = [bucket0, bucket1, bucket2, bucket3, bucket4, bucket5]

In [ ]:
bucket5.head(40)

In [27]:
bucket4.head(40)

productfamily_seller_standard  \
1580              evacuation des eaux de pluie   
1844                    chauffe eau electrique   
1888                                chaudieres   
3861                             gouttiere pvc   
1707                                couverture   
449                              kits d outils   
665                       secateurs et ciseaux   
4170                             gouttiere pvc   
3857                  etancheite de la toiture   
639                     brossettes de rechange   
772                      ventilateurs de table   
2460                               anti mousse   
710                          meuleuses d angle   
290                                   broyeurs   
730                                     mastic   
2331              evacuation des eaux de pluie   
2394              evacuation des eaux de pluie   
2332              evacuation des eaux de pluie   
1678                                chaudieres   
4053                             gouttiere pvc   
1577                                couverture   
2206      etancheite des toitures et terrasses   
831                          scies circulaires   
1833              evacuation des eaux de pluie   
3748                  etancheite de la toiture   
1933              evacuation des eaux de pluie   
4010                  etancheite de la toiture   
267                          scies circulaires   
3650                             gouttiere pvc   
1624                                couverture   
1734                                couverture   
69    aspirateurs balais et balais electriques   
779                      ponceuses excentrique   
3686                             gouttiere pvc   
2295                    chauffe eau electrique   
3790                             gouttiere pvc   
155                                ebrancheurs   
735                             scies   metaux   
687                          packs de batterie   
3692                             gouttiere pvc   

                            productseller_name_standard  \
1580             recuperateur d eau de pluie a enterrer   
1844  chauffe eau resistance steatite et anode titan...   
1888    chaudiere murale gaz a condensation isotwin  kw   
3861                        jonction a joint anthracite   
1707             tuile a douille belmont anthracite mat   
449   aeg mth   etnbsp petit chauffe eau hydraulique...   
665                        black and decker   ciseaux     
4170                 angle exterieur de gouttiere blanc   
3857                  etancheite toiture terre cuite  l   
639          aeg   embouts de brosse a dents electrique   
772   ryobi  f ventilateur sans fil one plus   hyper...   
2460         traitement anti mousses sika stop mousses    
710                  ryobi   kit meuleuse  v ryobi  sac   
290            ryobi  broyeur de vegetaux electrique  w   
730   sikaflex am auto marine   mastic colle carross...   
2331              tampon de sol en polypropylene  sable   
2394              tampon de sol en polypropylene  sable   
2332                           collier pvc gris   diam    
1678  chaudiere murale gaz a condensation themafast ...   
4053                    collecteur d eau pluviale sable   
1577                       panneau tuile pvc terracotta   
2206              bande d etancheite autocollante gris    
831               aeg  scie circulaire   w  multicolore   
1833               tampon de sol en polypropylene  gris   
3748                  etancheite toiture gris ciment  l   
1933                       collier developpe  sable pvc   
4010                    etancheite multi usages gris  l   
267   scie circulaire sans fil  v one  sans batterie...   
3650                crochet bandeau de gouttiere  sable   
1624                      tuile douille cplte signy rge   
1734               tuile douille cplte signy anthracite   
69        aeg    de    en  aspirateur a main  iron gris   
779                

In [ ]:
bucket3.head(40)

On voit là encore que les rapprochements deviennent imprécis lorsque la similitude de Jaccard tombe en-dessous de 0.15.

Voyons maintenant ce qui se passe quand on utilise les catégories vendeur les plus profondes (catégorie 4):

In [28]:
df_test = df2.loc[test_indices]
df_test2 = jaccard_matrix(df_test, "cat4", typo, "innoscape_ref_productfamily_seller" , "prod_fam", "similarity_cat4")
df_test2['max_sim_cat4'] = df_test2.apply(get_max_similarity, args=(["similarity_cat4"]), axis=1)
df_test2['argmax_sim_cat4'] = df_test2.apply(get_argmax_similarity, args=(["similarity_cat4"]), axis=1)
df_test2['best_match_cat4'] = df_test2.apply(get_category, args=('argmax_sim_cat4','max_sim_cat4'), axis=1)
matching_cat4 = df_test2[['productfamily_seller_standard','prodcategory4_seller_standard' ,'best_match_cat4', 'max_sim_cat4']]

In [ ]:
bucket0 = matching_cat4[matching_cat4['max_sim_cat4']==0].dropna()
bucket1 = matching_cat4.where((matching_cat4['max_sim_cat4']>0) & (matching_cat4['max_sim_cat4']<0.2)).dropna()
bucket2 = matching_cat4.where((matching_cat4['max_sim_cat4']>0.2) & (matching_cat4['max_sim_cat4']<0.4)).dropna()
bucket3 = matching_cat4.where((matching_cat4['max_sim_cat4']>0.4) & (matching_cat4['max_sim_cat4']<0.6)).dropna()
bucket4 = matching_cat4.where((matching_cat4['max_sim_cat4']>0.6) & (matching_cat4['max_sim_cat4']<0.8)).dropna()
bucket5 = matching_cat4.where((matching_cat4['max_sim_cat4']>0.8)).dropna()

buckets = [bucket0, bucket1, bucket2, bucket3, bucket4, bucket5]

In [ ]:
bucket1.head(40)

In [ ]:
bucket0.shape[0]

On observe des correspondances avec les catégories Innoscape dans près de 70% des cas. Cependant, les rapprochements sont moins précis qu'avec les familles produits. On peut envisager un seuil de similitude de 0.2-0.25. On observe par ailleurs que les rapprochements les plus pertinents semble être les suivants:
    1) rapprochement entre la famille produit et les familles produit Innoscape
    2) rapprochement entre la catégorie la plus profonde et les familles produit Innoscape
    3) rapprochement entre la description et les familles produit Innoscape
    
Quels résultats obtient-on en essayant successivement ces trois méthodes, et en retenant un rapprochement dès que la mesure de similitude dépasse le seuil voulu?

In [34]:
#on concatène les résultats obtenus par les trois méthodes dans un seul dataframe:
matching_descr.drop(['productfamily_seller_standard','productseller_name_standard'], axis=1, inplace=True)
matching_cat4.drop(['productfamily_seller_standard'], axis=1, inplace=True)
compare = pd.concat([matching_prod_fam, matching_descr, matching_cat4], axis=1)

In [38]:
def choose_match(row, threshold1, threshold2, threshold3):
    scores = row[['max_sim_prod_fam','max_sim_cat4','max_sim_descr']]
    matches = row[['best_match_prod_fam','best_match_cat4','best_match_descr']]
    
    if row['max_sim_prod_fam'] >= threshold1:
        return pd.Series([row['best_match_prod_fam'], row['max_sim_prod_fam']])
    elif row['max_sim_cat4'] >= threshold2:
        return pd.Series([row['best_match_cat4'], row['max_sim_cat4']])
    elif row['max_sim_descr'] >= threshold3:
        return pd.Series([row['best_match_descr'],row['max_sim_descr']])
    elif max(scores) > np.float64(0) :
        return pd.Series([matches.values[np.argmax(scores.values)],max(scores.values)])
    else:
        return pd.Series(["no match", 0.0])

In [40]:
toto=compare.copy()

In [41]:
toto.columns

Index(['productfamily_seller_standard', 'best_match_prod_fam',
       'max_sim_prod_fam', 'best_match_descr', 'max_sim_descr',
       'prodcategory4_seller_standard', 'best_match_cat4', 'max_sim_cat4'],
      dtype='object')

In [42]:
toto[['best_match','similarity']]=toto.apply(choose_match, args=(0.15,0.2,0.15), axis=1)

In [43]:
toto[['productfamily_seller_standard', 'best_match_prod_fam', 'best_match_descr', 'best_match_cat4', 'best_match', 'similarity']].head(50)

productfamily_seller_standard  \
284                          meuleuses droites   
1972                    chauffe eau electrique   
2268                   accessoires chauffe eau   
1505              evacuation des eaux de pluie   
1768                                couverture   
348                        tondeuses rotatives   
4173                             gouttiere pvc   
173                         climatiseurs split   
1529                                couverture   
2287              evacuation des eaux de pluie   
1968                                couverture   
1450                                couverture   
1808                                couverture   
2251                                chaudieres   
1466              evacuation des eaux de pluie   
1580              evacuation des eaux de pluie   
1844                    chauffe eau electrique   
2290              evacuation des eaux de pluie   
555                            scies sauteuses   
3704                  etancheite de la toiture   
3856                  etancheite de la toiture   
2414              evacuation des eaux de pluie   
2370              evacuation des eaux de pluie   
236                          perceuse visseuse   
128                                 agrafeuses   
1983      etancheite des toitures et terrasses   
2214                                couverture   
2141                                couverture   
49                          colles   carrelage   
585                         contacteurs   cles   
1749                   accessoires chauffe eau   
1531                                couverture   
784                           beches et pelles   
1774              evacuation des eaux de pluie   
2081                   accessoires chauffe eau   
2023                                couverture   
1888                                chaudieres   
1620              evacuation des eaux de pluie   
598       brise vue et pare soleil pour jardin   
102                        coussins chauffants   
120   aspirateurs balais et balais electriques   
1495              evacuation des eaux de pluie   
2348         chauffe eau instantane electrique   
159                                     mastic   
222                      ponceuses excentrique   
1874                                couverture   
2088              evacuation des eaux de pluie   
3861                             gouttiere pvc   
2198                    chauffe eau electrique   
2096                                chaudieres   

                                  best_match_prod_fam  \
284                                          no match   
1972                          chauffe eau electrique    
2268                         accessoires chauffe eau    
1505                                         no match   
1768                                       couverture   
348   tondeuses electriques  thermiques et autoportee   
4173             gouttiere  descente et accessoires     
173                                          no match   
1529                                       couverture   
2287                                         no match   
1968                                       couverture   
1450                                       couverture   
1808                                       couverture   
2251                                         no match   
1466                                         no match   
1580                                         no match   
1844                          chauffe eau electrique    
2290                                         no match   
555                                 scies electriques   
3704                         mastic colle etancheite    
3856                         mastic colle etancheite    
2414                                         no match   
2370                                         no match   
236         perceuse  perforateur et visseuse filaire   
128                                      

In [ ]:
for c in toto.columns:
    print("col_name: %s | value: %s | type: %s" %(c, str(toto.loc[1687,c]),str(type(toto.loc[1687,c]))))

In [ ]:
toto.loc[0].head()

In [ ]:
scores = toto.loc[0,['max_sim_prod_fam','max_sim_cat4','max_sim_descr']]
n = np.argmax(scores.values)
matches = toto.loc[0,['best_match_prod_fam','best_match_cat4','best_match_descr']]
ga = matches.values[n]
ga

In [ ]:
type(0.0)

In [ ]:
type(np.float64(0))